In [1]:
import embree
import numpy as np
from scipy import misc, optimize, fft as scipy_fft
import trimesh
from matplotlib import pyplot, colors, cm
from PIL import Image
from IPython.display import display

def sampling_lattice(res, pitch=1.0):
    """Return an array of shape (res,) containing the coordinates of a sampling lattice with 0 as one of the samples."""
    return scipy_fft.fftshift(scipy_fft.fftfreq(res).astype(np.float32) * (res * pitch))

def display_image(values, color_map=None, absolute=True, normalize=True, flip=False):
    """Display an array as an image using a color map. The image can be normalized, shifted and/or flipped."""
    values = np.abs(values) if absolute else values
    values = values * np.nan_to_num(1.0 / np.amax(values)) if normalize else values
    values = np.flip(values) if flip else values
    color_values = cm.get_cmap(color_map)(values, bytes=True)
    display(Image.fromarray(color_values).transpose(Image.ROTATE_90))

z_r = -17.0
z_a = 10.0
z_b = 30.0

size_r = 5.5
size_p = 5.0
size_a = size_r * z_a / abs(z_r) + size_p
size_b = size_r * z_b / abs(z_r) + size_p

n_r = 512
n_a = 45
n_b = 45

w_r = size_r / n_r
w_p = size_p
w_a = size_a / n_a
w_b = size_b / n_b

z_f = 100.0

In [2]:
model = trimesh.load_mesh('../models/teapot/teapot.obj')
if isinstance(model, trimesh.Scene):
    meshes = list(model.geometry.values())
if isinstance(model, trimesh.Trimesh):
    meshes = [model]

device = embree.Device()
scene = device.make_scene()
for mesh in meshes:
    geometry = device.make_geometry(embree.GeometryType.Triangle)
    vertex_buffer = geometry.set_new_buffer(embree.BufferType.Vertex, 0, embree.Format.Float3, 3 * np.dtype('float32').itemsize, mesh.vertices.shape[0])
    index_buffer = geometry.set_new_buffer(embree.BufferType.Index, 0, embree.Format.Uint3, 3 * np.dtype('uint32').itemsize, mesh.faces.shape[0])
    vertex_buffer[:] = (mesh.vertices - mesh.center_mass) * [1.0, 1.0, -1.0] + [0.0, 0.0, 600.0]
    index_buffer[:] = mesh.faces
    geometry.commit()
    scene.attach_geometry(geometry)
    geometry.release()
scene.commit()

x_r = sampling_lattice(n_r, w_r)
retina_samples = np.stack(np.broadcast_arrays(x_r[:, np.newaxis], x_r[np.newaxis, :], z_r), axis=-1).reshape(-1, 3)

rayhit = embree.RayHit1M(retina_samples.shape[0])
rayhit.org[:] = retina_samples
rayhit.dir[:] = -retina_samples
rayhit.tnear[:] = 0
rayhit.tfar[:] = np.inf
rayhit.flags[:] = 0
rayhit.prim_id[:] = embree.INVALID_GEOMETRY_ID
rayhit.geom_id[:] = embree.INVALID_GEOMETRY_ID

context = embree.IntersectContext()
scene.intersect1M(context, rayhit)

#print(rayhit.normal.shape)
#print(rayhit.uv.shape)
#print(rayhit.geom_id.shape)
#print(rayhit.prim_id.shape)

retinal_image = np.zeros((512*512, 4))

mask = rayhit.geom_id != embree.INVALID_GEOMETRY_ID

hit_uv = rayhit.uv[mask, np.newaxis, :]
texture_uv = meshes[0].visual.uv[meshes[0].faces[rayhit.prim_id[mask], :], :]
interpolated_uv = (1.0 - hit_uv[..., 0] - hit_uv[..., 1]) * texture_uv[:, 0, :] + hit_uv[..., 0] * texture_uv[:, 1, :] + hit_uv[..., 1] * texture_uv[:, 2, :]

print(mask.shape)
print(hit_uv.shape)
print(texture_uv.shape)
print(interpolated_uv.shape)

with Image.open('../models/teapot/default.png') as texture:
    retinal_image[mask] = trimesh.visual.color.uv_to_color(interpolated_uv, texture)

retinal_image = np.flip(retinal_image.reshape(512, 512, 4), (0, 1))

display(Image.fromarray(retinal_image).transpose(Image.ROTATE_90))

#retina_img = np.sum(uv * [0.5, 1.0], axis=-1).reshape(512, 512)
#display_image(retina_img, flip=True)

#retina_img = rayhit.tfar.reshape(512, 512)
#retina_img[np.isinf(retina_img)] = np.amin(retina_img)
#retina_img = retina_img - np.amin(retina_img)
#display_image(retina_img, flip=True)

#retina_img = np.sum(rayhit.uv * [0.5, 1.0], axis=-1).reshape(512, 512)
#display_image(retina_img, flip=True)

scene.release()
device.release()

(262144,)
(45311, 1, 2)
(45311, 3, 2)
(45311, 2)


TypeError: Cannot handle this data type: (1, 1, 4), <f8